<a href="https://colab.research.google.com/drive/1XlUIBNyaKLApsZLT3KQuRR_GqvcLe4ck?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Training StyleGAN2 on the cropped dataset in Google CoLab 

![sample_images](https://raw.githubusercontent.com/mahdi-darvish/GAN-augmented-pet-classifier/main/Figures/thumb.jpg)

 ### Google Drive setups

One of the main requirements for starting is google drive. We're going to use G-DRIVE to store our training data and trained neural networks. we built some folders with the path of follows:
```
content/drive/MyDrive/gan 
``` 

### Managing GPU

There are two options for training GANs with Google Colab, Google Colab Free or Pro. We will go with the pro one because it has advantages like better GPU, longer runtime, and timeouts, and most important, it will not disconnect before 24 hours. In using the free version of google colab, you should make sure that you run the notebook with GPU runtime. For doing this, go to "Runtime" in the bar and select the "change the runtime type" option. Then, put the Hardware accelerator on GPU and save it.

In [11]:
!nvidia-smi

Thu May 19 07:55:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Steps of the code

In [12]:
import shutil
import os
import os
from tqdm.notebook import tqdm
from PIL import Image
from os import listdir
from PIL import Image
import os, sys
import os

Connecting Colab to GDRIVE, to save the snapshots there

In [13]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


Creating needful directories in GDRIVE

In [14]:
!mkdir drive/MyDrive/
!mkdir drive/MyDrive/gans_training
!mkdir drive/MyDrive/gans_training/experiments

mkdir: cannot create directory ‘drive/MyDrive/’: File exists


Setting the version of the torch, torchvision,  torchaudio to become compatible with Style-GAN2ada

In [15]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


Installing NVIDIA StyleGAN2 ADA PyTorch

In [6]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 1.98 MiB/s, done.
Resolving deltas: 100% (57/57), done.
     |████████████████████████████████| 108 kB 4.1 MB/s 


Download [the __cropped__](https://github.com/mahdi-darvish/GAN-augmented-pet-classifier#dataset) dataset

Extract the images

Unzipping the training images in "images" folder

In [22]:
!unzip  drive/MyDrive/images -d drive/MyDrive/gans_training/

Streaming output truncated to the last 5000 lines.
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_90.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_91.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_92.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_93.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_94.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_95.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_96.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_97.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_98.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/Egyptian_Mau_99.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/english_cocker_spaniel_1.jpg  
  inflating: drive/MyDrive/gans_training/images - Copy/english_cocker_spaniel_10.jpg  
  infl

Number of total training images and establishig the exact path of images

In [43]:
!ls drive/MyDrive/gans_training/images | wc -l

7394


This code takes a python file and converts the first given path to tensors into the second given direction. We can face an error in this part in conditions like inconsistent types and variant sizes of images. We convert all the images into jpg type and resize them to 128 x 128 pixels.

In [42]:
IMAGE_PATH = 'drive/MyDrive/gans_training/images'
files = [f for f in listdir(IMAGE_PATH) if os.path.isfile(os.path.join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
    file2 = os.path.join(IMAGE_PATH,file)
    img = Image.open(file2)
    sz = img.size
    if base_size and sz!=base_size:
      print(f"Inconsistant size: {file2}")
    elif img.mode!='RGB':
      print(f"Inconsistant color format: {file2}")
    else:
      base_size = sz


  0%|          | 0/7394 [00:00<?, ?it/s]

Making all the images to the exact dimensions and color depth

Converting all the images to jpg type

In [41]:
path = "drive/MyDrive/gans_training/images"

for item in os.listdir(path):
    im = Image.open(path + '/' +  item)
    if im.mode != "RGB" :
        im = im.convert("RGB")
    imResize = im.resize((128,128))
    imResize.save(path + '/' +  item )



To clear out the newly created dataset in case something went wrong 

In [ ]:
!rm -rf /root/.cache/torch_extensions/*

Creating our dataset.json file so that we can train a conditional GAN

In [44]:
import json
source = "drive/MyDrive/gans_training/images"
destination = "drive/MyDrive/gans_training/images"
files_list = os.listdir(source)
data_dict = {}
data_dict['labels'] = []
curr_class = None
class_counter = -1
for files in sorted(files_list):
  x = files.rsplit('_', 1)[0]
  if x != curr_class:
    curr_class = x
    class_counter += 1
  data_dict['labels'].append([files, class_counter])
with open(os.path.join(destination, 'dataset.json'), 'w') as outfile:
  json.dump(data_dict, outfile)



In [51]:
!rm -rf /content/drive/MyDrive/gans_training/dataset/
!mkdir /content/drive/MyDrive/gans_training/dataset/

In [52]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/gans_training/images/ --dest /content/drive/MyDrive/gans_training/dataset/

100% 7394/7394 [01:30<00:00, 82.15it/s]


### Perform Initial Training

In [10]:
# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/gans_training/experiments"
DATA = "/content/drive/MyDrive/gans_training/dataset"
SNAP = 20

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Aborted!
^C


### Resume Training

Removing the last trained network into the experiment folder for continuing the training

In [2]:
!mkdir drive/MyDrive/gans_training/models

mkdir: cannot create directory ‘drive/MyDrive/gans_training/models’: File exists


In [4]:
!cp /content/drive/MyDrive/gans_training/experiments/00000-dataset-auto1/network-snapshot-000000.pkl drive/MyDrive/gans_training/models

In [9]:
EXPERIMENTS = "/content/drive/MyDrive/gans_training/models/"
NETWORK = "network-snapshot-000000.pkl"
RESUME = os.path.join(EXPERIMENTS, NETWORK)
DATA = "/content/drive/MyDrive/gans_training/dataset"
SNAP = 20

cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 20,
  "network_snapshot_ticks": 20,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/gans_training/dataset",
    "use_labels": false,
    "max_size": 4458,
    "xflip": false,
    "resolution": 128
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Copyright 2021 by [MIT license](https://github.com/mahdi-darvish/GAN-augmented-pet-classifier/blob/main/LICENSE)